<a href="https://colab.research.google.com/github/edri-avichai/Practical-Deep-Learning-Workshop-Assignment-4/blob/main/one%20hot%20bias%20detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost
!pip install shap
import pprint
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from tabulate import tabulate
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
import shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 7.3 MB/s eta 0:00:00


In [2]:
# Read in the data
categorical_threshold = 10
tpr_threshold = 0.6 # small tpr is bad
fpr_threshold = 0.4 # small fpr is good
sample_is_bias_threshold = 1 # minimum number of features that are biased in a sample
odds_difference_threshold = 0.02 # small odds difference is good
google_colab = True

In [3]:
def get_categorical_features(dataset, threshold ):
    return [col for col in dataset.columns if dataset[col].dtype == 'object' or dataset[col].nunique() <= threshold]

In [4]:
def one_hot_encode_categorical_features(dataset, categorical_features, dataset_name):

    # Create a copy of the original dataset to avoid modifying the input dataset
    encoded_dataset = dataset.copy()
    new_columns = []

    # Iterate through each categorical feature and apply one-hot encoding
    for feature in categorical_features:
        # Handle NULL values by filling them with a unique category label (e.g., 'NULL')
        encoded_dataset[feature] = encoded_dataset[feature].fillna('NULL')

        # Perform one-hot encoding using Pandas' get_dummies function
        encoded_feature = pd.get_dummies(encoded_dataset[feature], prefix=feature)
        new_columns.extend(encoded_feature.columns)

        # Concatenate the one-hot encoded feature back to the dataset and drop the original feature
        encoded_dataset = pd.concat([encoded_dataset, encoded_feature], axis=1)
        encoded_dataset.drop(columns=[feature], inplace=True)
    #save onehot dats_set
    encoded_dataset.to_csv(f"one_hot_{dataset_name}.csv", index=False)
    return encoded_dataset, new_columns

In [5]:
if not google_colab:
  adult_base = pd.read_csv("D:\master\datasets\\adult.csv")
  compass_base = pd.read_csv('D:\master\datasets\compas-scores-two-years_v1.csv')
  law_base = pd.read_csv('D:\master\datasets\\law_school.csv')
  # german_base = pd.read_csv('D:\master\datasets\\german.csv')
  # wids_base = pd.read_csv('D:\master\datasets\\wids.csv')
  diabetes_base = pd.read_csv('D:\master\datasets\\diabetes-clean.csv')
else:
  from google.colab import drive

  # This will prompt for authorization.
  drive.mount('/content/drive')
  adult_base = pd.read_csv("/content/drive/MyDrive/master/datasets/adult.csv")
  compass_base = pd.read_csv('/content/drive/MyDrive/master/datasets/compas-scores-two-years_v1.csv')
  law_base = pd.read_csv('/content/drive/MyDrive/master/datasets/law_school.csv')
  # german_base = pd.read_csv('D:\master\datasets\\german.csv')
  # wids_base = pd.read_csv('D:\master\datasets\\wids.csv')
  diabetes_base = pd.read_csv('/content/drive/MyDrive/master/datasets/diabetes-clean.csv')


Mounted at /content/drive


In [6]:
compass_base = compass_base[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count',
               'days_b_screening_arrest', 'decile_score', 'two_year_recid']]
compass_base = compass_base[((compass_base['days_b_screening_arrest'] <= 30) &
                             (compass_base['days_b_screening_arrest'] >= -30) &
                             # (compass['is_recid'] != -1) &
                             (compass_base['c_charge_degree'] != 'O') &
                             (compass_base['score_text'] != 'N/A')
                             )]
# compass_cat =[ 'sex','age_cat', 'race', 'c_charge_degree', 'score_text']
compass_cat = get_categorical_features(compass_base, categorical_threshold)
compass_target = 'two_year_recid'
compass_cat.remove(compass_target)
compass,compass_one_hot_col = one_hot_encode_categorical_features(compass_base, compass_cat, "compass")

In [7]:
adult  = adult_base[['age', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country','income']]
adult_target = 'income'
adult_cat = get_categorical_features(adult, categorical_threshold)
adult_cat.remove(adult_target)
adult[adult_target] = adult[adult_target].apply(lambda x: 0 if x == '<=50K' else 1)
adult,adult_one_hot_col = one_hot_encode_categorical_features(adult, adult_cat, "adult")



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
law_target = 'pass_bar'
law_cat = get_categorical_features(law_base, categorical_threshold)
law_cat.remove(law_target)
law,law_one_hot_col = one_hot_encode_categorical_features(law_base, law_cat, "law")
print(law[law_target].value_counts())

1.0    18505
0.0     2293
Name: pass_bar, dtype: int64


In [9]:
diabetes_target = 'readmitted'
diabetes_cat = get_categorical_features(diabetes_base, categorical_threshold)
diabetes_cat.remove(diabetes_target)
diabetes,diabetes_one_hot_col = one_hot_encode_categorical_features(diabetes_base, diabetes_cat, "diabetes")
diabetes[diabetes_target] = diabetes[diabetes_target].apply(lambda x: 0 if x == '<30' else 1)

In [10]:
def one_csv_of_values_count_in_dataset(dataset, dataset_name):
    dataset_value_count = pd.DataFrame(columns=['feature', '0', '1'])
    for col in dataset.columns:
        value_counts = dataset[col].value_counts()
        count_0 = value_counts[0] if 0 in value_counts.index else 0
        count_1 = value_counts[1] if 1 in value_counts.index else 0
        row_data = {'feature': col, '0': count_0, '1': count_1}
        dataset_value_count = pd.concat([dataset_value_count, pd.DataFrame([row_data])], ignore_index=True)
    dataset_value_count.to_csv(f"values_count_{dataset_name}.csv", index=False)
one_csv_of_values_count_in_dataset(compass, "compass")
one_csv_of_values_count_in_dataset(adult, "adult")


In [11]:
def print_metrics(y_gt, y_pred):
    cm = confusion_matrix(y_gt, y_pred)
    if cm.shape !=(2,2):
        return None
    if cm.shape == (1, 1):
        if 1 == y_gt.iloc[0] and 1 == y_pred.iloc[0]:
            tn, fp, fn, tp = 0, 0, 0, 1
        else:
            tn, fp, fn, tp = 1, 0, 0, 0
    else:
        tn, fp, fn, tp = confusion_matrix(y_gt, y_pred).ravel()
    accuracy = (tp+tn)/(tp+tn+fn+fp)
    # precision = tp/(tp+fp)
    if tp + fp == 0:
        precision = 0  # Handle the case where denominator is zero
    else:
        precision = tp / (tp + fp)
    # recall = tp/(tp+fn)
    if tp + fn == 0:
        recall = 0  # Handle the case where denominator is zero
    else:
        recall = tp / (tp + fp)
    if fp+tn == 0:
        fpr = 0  # Handle the case where denominator is zero
    else:
        fpr = fp/(fp+tn)

    if tp + fn == 0:
        tpr = 0  # Handle the case where denominator is zero
    else:
        tpr = tp /(tp + fn)


    fn_fp = fn+fp

    # print('conf matrix:\n', confusion_matrix(y_gt, y_pred))
    metrics_table = [ ["TN"," FP", "FN","TP","Accuracy"],
        [ tn, fp, fn, tp, accuracy],
        ["Precision","Recall", "False Positive Rate (FPR)","True Positive Rate (TPR)","Sum of FN and FP"],
        [ precision, recall, fpr, tpr, fn_fp]]

    # Print the metrics table
    # print(tabulate(metrics_table, tablefmt="grid"))

    #returm fpr, tpr, accuracy
    return {"fpr": fpr,"tpr": tpr, "accuracy": accuracy}

# bias metric - equal opportunity difference
def opportunity_diff_tpr(tpr1, tpr2):
    return abs(tpr1-tpr2)

# bias metric - equal opportunity difference
def opportunity_diff_fpr(fpr1, fpr2):
    return abs(fpr1-fpr2)

# bias metric - average absolute odds difference
def odds_diff(tpr1, tpr2, fpr1, fpr2):
    return 0.5*(abs(tpr2-tpr1) + abs(fpr2-fpr1))

# bias metric - statistical parity difference
def parity_diff(y_pred_0, y_pred_1):
    return abs(len(y_pred_0)/(len(y_pred_0)+len(y_pred_1)) - len(y_pred_1)/(len(y_pred_1)+len(y_pred_0)))


    # bias metric - Disparate impact
def disparate_impact(y_pred_0, y_pred_1):
    if len(y_pred_0) == 0 or len(y_pred_1) == 0:
        return 0

    return min((len(y_pred_0)/len(y_pred_0 + y_pred_1)) / (len(y_pred_1)/len(y_pred_1+len(y_pred_0)))
    ,(len(y_pred_1)/len(y_pred_0+y_pred_1)) / (len(y_pred_0)/len(y_pred_1+len(y_pred_0))))

In [12]:
def bias_in_single_column(dataset, column_name, target):
    # print('bias in column: ', column_name)
    one_value = dataset[dataset[column_name] == 1]
    zero_value = dataset[dataset[column_name] == 0]

    one_metrics = print_metrics(one_value[target], one_value['y_pred'])
    if one_metrics is None:
        return None
    zero_metrics = print_metrics(zero_value[target], zero_value['y_pred'])
    if zero_metrics is None:
        return None
    one_value_majority = one_value[target].value_counts().idxmax()
    zero_value_majority = zero_value[target].value_counts().idxmax()
    o_d_t = opportunity_diff_tpr(one_metrics['tpr'], zero_metrics['tpr'])
    o_d_f = opportunity_diff_fpr(one_metrics['fpr'], zero_metrics['fpr'])
    o_d = odds_diff(one_metrics['tpr'], zero_metrics['tpr'], one_metrics['fpr'], zero_metrics['fpr'])
    p_d = parity_diff(one_value['y_pred'], zero_value['y_pred'])
    # d_i = disparate_impact(one_value['y_pred'], zero_value['y_pred'])

    if o_d > odds_difference_threshold:
        bias_dict = {'equal opportunity for tpr: (close 0)': o_d_t,
                      'equal opportunity for fpr: (close 0)': o_d_f,
                      'odds difference: (close 0)': o_d,
                      'statistical parity difference: (close 0)': p_d,
                      # 'disparate impact: (close 1)': disparate_impact_
                      f'tpr for 1': one_metrics["tpr"],
                    f'fpr for 1': one_metrics["fpr"],
                    f'tpr for 0': zero_metrics["tpr"],
                    f'fpr for 0 ': zero_metrics["fpr"],
                     f'majority in 1': one_value_majority,
                     f'majority in 0': zero_value_majority,
                      }
        if one_metrics['tpr'] < tpr_threshold:
                bias_dict["privilege in one"] = False
        else:
                bias_dict["privilege in one"] = True
        if zero_metrics['tpr'] < tpr_threshold:
                bias_dict["privilege in zero"] = False
        else:
                bias_dict["privilege in zero"] = True
        # pprint.pprint(bias_dict)


        return bias_dict



In [33]:
def bias_in_dataset(train_set, target ,categorical_features):
    bias_dict = {}
    train, val = train_test_split(train_set, test_size=0.25, random_state=42,stratify=train_set[target])
    model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=5, loss_function='Logloss', verbose= False)
    model.fit(train.drop(columns=[target]), train[target], cat_features=categorical_features)
    val['y_pred'] = model.predict(val.drop(columns=[target]))
    print(accuracy_score(val['y_pred'],val[target]))
    for col in categorical_features:

        col_bias = bias_in_single_column(val, col, target)
        if col_bias is not None:
            bias_dict[col] = col_bias
    return bias_dict



In [14]:
def bias_features_using_shap(train_set, target ,categorical_features):
    train, val = train_test_split(train_set, test_size=0.25, random_state=42,stratify=train_set[target])
    model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=5, loss_function='Logloss', verbose=True)
    model.fit(train.drop(columns=[target]), train[target], cat_features=categorical_features)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer(train.drop(columns=[target]), train[target])
    shap.summary_plot(shap_values, train.drop(columns=[target]))
    shap.summary_plot(shap_values, train.drop(columns=[target]), plot_type="bar")



In [34]:
def import_csv_summary(bias_dict, dataset_name):
    bias_summary = []
    columns = ["dataset", "feature"]
    columns.extend(list(bias_dict[list(bias_dict.keys())[0]].keys()))
    for feature in bias_dict.keys():
        bias_summary.append([dataset_name, feature])
        for metric in bias_dict[feature].keys():
            bias_summary[-1].append(bias_dict[feature][metric])
    bias_summary = pd.DataFrame(bias_summary, columns=columns)
    bias_summary.to_csv(f"bias_summary_{dataset_name}.csv", index=False)

In [35]:
def is_sample_biased(sample,unpriv_cols,bias_dict,bias_threshold):
    biased_cols = 0
    for col in unpriv_cols:
        if sample[col] == 1 and not bias_dict[col]["privilege in one"] and bias_dict[col]["majority in 1"] == sample["pred"]:
            biased_cols += 1
        elif sample[col] == 0 and not bias_dict[col]["privilege in zero"] and bias_dict[col]["majority in 0"] == sample["pred"]:
            biased_cols += 1
    if biased_cols >= bias_threshold:
        return True
    return False

In [36]:
def identify_bias_samples(test, bias_dict, biased_cols):
    list_of_biased_samples = []

    for index, sample in test.iterrows():
        if is_sample_biased(sample, biased_cols, bias_dict):
            list_of_biased_samples.append(index)
    return list_of_biased_samples


In [37]:
def calc_test_bias_no_bias(test, model,target,bias_dict,bias_threshold):
    biased_cols =[]

    for feature in bias_dict.keys():
        if not bias_dict[feature]["privilege in one"] or not bias_dict[feature]["privilege in zero"]:
            biased_cols.append(feature)
    test["pred"] = model.predict(test.drop(target, axis=1))
    original_accuracy = accuracy_score(test[target], test["pred"])
    print(f"original accuracy: {original_accuracy}")
    test["is_bias"] = test.apply(lambda row: is_sample_biased(row, biased_cols,bias_dict,bias_threshold), axis=1)
    test_sample_with_bias = test[test["is_bias"]==True]
    test_sample_without_bias = test[test["is_bias"]==False]
    accuracy_with_bias = accuracy_score(test_sample_with_bias[target], test_sample_with_bias["pred"])

    # Calculate accuracy for samples without bias
    accuracy_without_bias = accuracy_score(test_sample_without_bias[target], test_sample_without_bias["pred"])
    print('number of samples: ', len(test))
    print('number of samples with bias: ', len(test_sample_with_bias))
    print('number of samples without bias: ', len(test_sample_without_bias))
    print('percentage of samples with bias: ', len(test_sample_with_bias)/len(test))
    return accuracy_with_bias, accuracy_without_bias


In [38]:
compass_train, compass_test = train_test_split(compass, test_size=0.2, random_state=42,stratify=compass[compass_target])
compass_bias_dict = bias_in_dataset(compass_train, compass_target, compass_one_hot_col)
import_csv_summary(compass_bias_dict, "compass")
compass_infer_model = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=42)
compass_infer_model.fit(compass_train.drop(columns=[compass_target]), compass_train[compass_target])
compass_accuracy_with_bias, compass_accuracy_without_bias = calc_test_bias_no_bias(compass_test, compass_infer_model, compass_target, compass_bias_dict, sample_is_bias_threshold)
print(f"compass accuracy with bias: {compass_accuracy_with_bias}")
print(f"compass accuracy without bias: {compass_accuracy_without_bias}")


0.659919028340081
original accuracy: 0.6866396761133603
number of samples:  1235
number of samples with bias:  831
number of samples without bias:  404
percentage of samples with bias:  0.6728744939271255
compass accuracy with bias: 0.6787003610108303
compass accuracy without bias: 0.7029702970297029


In [39]:
adult_train, adult_test = train_test_split(adult, test_size=0.2, random_state=42,stratify=adult[adult_target])
adult_bias_dict = bias_in_dataset(adult_train, adult_target, adult_one_hot_col)
import_csv_summary(adult_bias_dict, "adult")
adult_infer_model = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=42)
adult_infer_model.fit(adult_train.drop(columns=[adult_target]), adult_train[adult_target])
adult_accuracy_with_bias, adult_accuracy_without_bias = calc_test_bias_no_bias(adult_test, adult_infer_model, adult_target, adult_bias_dict,4)
print(f"adult accuracy with bias: {adult_accuracy_with_bias}")
print(f"adult accuracy without bias: {adult_accuracy_without_bias}")

0.8668611793611793


invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars


original accuracy: 0.8238906801781053
number of samples:  6513
number of samples with bias:  5470
number of samples without bias:  1043
percentage of samples with bias:  0.8398587440503608
adult accuracy with bias: 0.8460694698354662
adult accuracy without bias: 0.7075743048897412


In [42]:
law_train, law_test = train_test_split(law, test_size=0.20, random_state=42,stratify=law[law_target])
law_bias_dict = bias_in_dataset(law_train, law_target, law_one_hot_col)
import_csv_summary(law_bias_dict, "law")
law_infer_model = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=42)
law_infer_model.fit(law_train.drop(columns=[law_target]), law_train[law_target])
law_accuracy_with_bias, law_accuracy_without_bias = calc_test_bias_no_bias(law_test, law_infer_model, law_target, law_bias_dict, sample_is_bias_threshold)
print(f"law accuracy with bias: {law_accuracy_with_bias}")
print(f"law accuracy without bias: {law_accuracy_without_bias}")

0.8995192307692308
original accuracy: 0.8966346153846154
number of samples:  4160
number of samples with bias:  0
number of samples without bias:  4160
percentage of samples with bias:  0.0
law accuracy with bias: nan
law accuracy without bias: 0.8966346153846154


Mean of empty slice.
invalid value encountered in double_scalars


In [41]:
diabetes_train,diavitic_test = train_test_split(diabetes, test_size=0.2, random_state=42, stratify=diabetes[diabetes_target])
diabetes_bias_dict = bias_in_dataset(diabetes_train, diabetes_target, diabetes_one_hot_col)
import_csv_summary(diabetes_bias_dict, "diabetes")
diabetes_infer_model = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)
diabetes_infer_model.fit(diabetes_train.drop(columns=[diabetes_target]), diabetes_train[diabetes_target])
diabetes_accuracy_with_bias, diabetes_accuracy_without_bias = calc_test_bias_no_bias(diavitic_test, diabetes_infer_model, diabetes_target, diabetes_bias_dict, sample_is_bias_threshold)
print(f"diabetes accuracy with bias: {diabetes_accuracy_with_bias}")
print(f"diabetes accuracy without bias: {diabetes_accuracy_without_bias}")


0.762878705020234


invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars
invalid value encountered in long_scalars


original accuracy: 0.7579569069233293
number of samples:  9143
number of samples with bias:  48
number of samples without bias:  9095
percentage of samples with bias:  0.005249917970031718
diabetes accuracy with bias: 0.7708333333333334
diabetes accuracy without bias: 0.7578889499725123
